In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [147]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [148]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['saved_library_scope']
redirect_uri = spotify_creds['saved_library_redirect_url']

In [149]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [150]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [151]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [152]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [87]:
def get_saved_library():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [88]:
df = get_saved_library()

In [116]:
songs = sp.current_user_saved_tracks(offset=0)
songs['items'][0]['track']['external_urls']

{'spotify': 'https://open.spotify.com/track/2XKsHHNCtKqk9cF35TRFyC'}

In [120]:
str1 = songs['items'][0]['track']['external_urls']['spotify']

In [153]:
scope = 'user-top-read'
redirect_uri = 'http://localhost:8885/callback'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [154]:
sp = spotipy.Spotify(auth=token)

In [155]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['top_artist_scope']
redirect_uri = spotify_creds['top_artist_redirect_url']


sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [156]:
a = sp.current_user_top_artists()

In [158]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['recently_played_scope']
redirect_uri = spotify_creds['recently_played_redirect_url']


sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)


def get_recently_played():

    df_saved_tracks = pd.DataFrame()
    track_list = ''
    played_ts_list = []
    artist_list = []
    title_list = []
    track_url_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True


    songs = sp.current_user_recently_played(limit=50)
    for song in songs['items']:
        #join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
        #get the time when the song was added
        played_ts_list.append(song['played_at'])
        #get the title of the song
        title_list.append(song['track']['name'])
        #get popularity
        popularity_list.append(song['track']['popularity'])
        # get album cover
        album_cover_url = song['track']['album']['images'][0]['url']
        album_cover_list.append(album_cover_url)
        # get track list
        track_url = song['track']['external_urls']['spotify']
        track_url = track_url.split('/')[-1]
        track_url_list.append(track_url)
        #get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
        track_list = ''
        #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['played_at'] = played_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    df_saved_tracks['url'] = track_url_list
    
    return df_saved_tracks